In [1]:
!pip install kaggle
!mkdir -p ~/.kaggle


In [ ]:
%%writefile ~/.kaggle/kaggle.json
{
  "username": "rashvin20",
  "key": "replace your key here"
}


Writing /root/.kaggle/kaggle.json


In [3]:
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d moltean/fruits

Dataset URL: https://www.kaggle.com/datasets/moltean/fruits
License(s): CC-BY-SA-4.0
 99% 4.73G/4.78G [01:42<00:01, 27.3MB/s]
100% 4.78G/4.78G [01:42<00:00, 50.2MB/s]


In [5]:
!unzip fruits.zip -d fruits_dataset/

Streaming output truncated to the last 5000 lines.
  inflating: fruits_dataset/fruits-360_original-size/fruits-360-original-size/Validation/Quince 4/r0_21.jpg  
  inflating: fruits_dataset/fruits-360_original-size/fruits-360-original-size/Validation/Quince 4/r0_213.jpg  
  inflating: fruits_dataset/fruits-360_original-size/fruits-360-original-size/Validation/Quince 4/r0_217.jpg  
  inflating: fruits_dataset/fruits-360_original-size/fruits-360-original-size/Validation/Quince 4/r0_221.jpg  
  inflating: fruits_dataset/fruits-360_original-size/fruits-360-original-size/Validation/Quince 4/r0_225.jpg  
  inflating: fruits_dataset/fruits-360_original-size/fruits-360-original-size/Validation/Quince 4/r0_229.jpg  
  inflating: fruits_dataset/fruits-360_original-size/fruits-360-original-size/Validation/Quince 4/r0_233.jpg  
  inflating: fruits_dataset/fruits-360_original-size/fruits-360-original-size/Validation/Quince 4/r0_237.jpg  
  inflating: fruits_dataset/fruits-360_original-size/fruits-36

In [6]:
!pip install -q wandb torchviz fvcore grad-cam scikit-learn graphviz
!apt-get -q install graphviz



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 89.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Reading package lists...
Building dependency tree...
Reading state information...
graphviz is already the newest version (2.42.2-6ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [13]:
import os, random
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from PIL import Image

from sklearn.metrics import f1_score, confusion_matrix, ConfusionMatrixDisplay
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
from fvcore.nn import FlopCountAnalysis
from torchviz import make_dot

import wandb

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Running on:", device)

os.makedirs("/content/outputs", exist_ok=True)


Running on: cuda


In [14]:
wandb.login()

wandb.init(
    project="vision-cw",
    name="cnn_noaugmneted",
    config={
        "dataset": "Fruits-360",
        "augmentation": True,
        "epochs": 30,
        "batch_size": 32,
        "lr": 1e-3
    }
)


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [ ]:
train_path = "/content/fruits_dataset/fruits-360_original-size/fruits-360-original-size/Training"
val_path   = "/content/fruits_dataset/fruits-360_original-size/fruits-360-original-size/Validation"


train_transform_no_aug = transforms.Compose([
    transforms.Resize((100,100)),
    transforms.ToTensor()
])

val_transform = transforms.Compose([
    transforms.Resize((100,100)),
    transforms.ToTensor()
])

train_dataset_full = datasets.ImageFolder(train_path, transform=train_transform_no_aug)
val_dataset_full   = datasets.ImageFolder(val_path, transform=val_transform)

num_classes = len(train_dataset_full.classes)
print("Classes:", num_classes)

NUM_TRAIN = 10000
NUM_VAL   = 2000

train_idx = random.sample(range(len(train_dataset_full)), NUM_TRAIN)
val_idx   = random.sample(range(len(val_dataset_full)), NUM_VAL)

train_dataset = Subset(train_dataset_full, train_idx)
val_dataset   = Subset(val_dataset_full, val_idx)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False)

print(f"Training images (no aug): {len(train_dataset)}")
print(f"Validation images:        {len(val_dataset)}")


Classes: 116
Training images (no aug): 10000
Validation images:        2000


In [ ]:

import os, random
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from PIL import Image

from sklearn.metrics import f1_score, confusion_matrix, ConfusionMatrixDisplay
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image

import wandb

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Running on:", device)

OUT_DIR = "/content/pretrained"
os.makedirs(OUT_DIR, exist_ok=True)

wandb.init(
    project="vision-cw",
    name="resnet18_pretrained_fruits360"
)


train_path = "/content/fruits_dataset/fruits-360_original-size/fruits-360-original-size/Training"
val_path   = "/content/fruits_dataset/fruits-360_original-size/fruits-360-original-size/Validation"


train_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std =[0.229, 0.224, 0.225]
    )
])

val_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std =[0.229, 0.224, 0.225]
    )
])


train_full = datasets.ImageFolder(train_path, transform=train_transform)
val_full   = datasets.ImageFolder(val_path, transform=val_transform)

num_classes = len(train_full.classes)
print("Classes:", num_classes)

NUM_TRAIN = 10000
NUM_VAL   = 2000

train_idx = random.sample(range(len(train_full)), NUM_TRAIN)
val_idx   = random.sample(range(len(val_full)), NUM_VAL)

train_dataset = Subset(train_full, train_idx)
val_dataset   = Subset(val_full, val_idx)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False)

print(f"Training images (pretrained): {len(train_dataset)}")
print(f"Validation images:            {len(val_dataset)}")

wandb.log({
    "train_images": len(train_dataset),
    "val_images": len(val_dataset),
    "num_classes": num_classes
})

#load pretarined model
model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)

# Replace classification head
model.fc = nn.Linear(model.fc.in_features, num_classes)

model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

EPOCHS = 20
best_val_loss = float("inf")
patience, wait = 5, 0

train_losses, val_losses = [], []
train_accs, val_accs = [], []

#training happen here
for epoch in range(EPOCHS):
    model.train()
    correct, total, running_loss = 0, 0, 0

    for x, y in tqdm(train_loader):
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        correct += (out.argmax(1) == y).sum().item()
        total += y.size(0)

    train_loss = running_loss / len(train_loader)
    train_acc = correct / total

    model.eval()
    v_loss, v_correct, v_total = 0, 0, 0
    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(device), y.to(device)
            out = model(x)
            v_loss += criterion(out, y).item()
            v_correct += (out.argmax(1) == y).sum().item()
            v_total += y.size(0)

    val_loss = v_loss / len(val_loader)
    val_acc = v_correct / v_total

    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accs.append(train_acc)
    val_accs.append(val_acc)

    wandb.log({
        "epoch": epoch + 1,
        "train_loss": train_loss,
        "val_loss": val_loss,
        "train_acc": train_acc,
        "val_acc": val_acc
    })

    print(f"Epoch {epoch+1}: TrainAcc={train_acc:.3f} ValAcc={val_acc:.3f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), f"{OUT_DIR}/best_model_pretrained.pth")
        wait = 0
    else:
        wait += 1
        if wait >= patience:
            break

#plotting code
plt.figure(figsize=(10,4))
plt.subplot(1,2,1); plt.plot(train_losses); plt.plot(val_losses); plt.title("Loss")
plt.subplot(1,2,2); plt.plot(train_accs); plt.plot(val_accs); plt.title("Accuracy")
plt.tight_layout()
plt.savefig(f"{OUT_DIR}/training_curves.jpg", dpi=300)
plt.close()

wandb.log({"training_curves": wandb.Image(f"{OUT_DIR}/training_curves.jpg")})

#validating the model
model.load_state_dict(torch.load(f"{OUT_DIR}/best_model_pretrained.pth", map_location=device))
model.eval()

#get f1
y_true, y_pred = [], []
with torch.no_grad():
    for x, y in val_loader:
        preds = model(x.to(device)).argmax(1)
        y_true.extend(y.numpy())
        y_pred.extend(preds.cpu().numpy())

f1 = f1_score(y_true, y_pred, average="macro")
print("Macro F1:", f1)
wandb.log({"macro_f1": f1})

cm = confusion_matrix(y_true, y_pred, normalize="true")
ConfusionMatrixDisplay(cm).plot(cmap="Blues")
plt.savefig(f"{OUT_DIR}/confusion_matrix.jpg", dpi=300)
plt.close()

wandb.log({"confusion_matrix": wandb.Image(f"{OUT_DIR}/confusion_matrix.jpg")})


img_path = val_full.imgs[val_idx[0]][0]
img = Image.open(img_path).resize((224,224)).convert("RGB")
img_np = np.array(img) / 255.0
tensor = val_transform(img).unsqueeze(0).to(device)

cam = GradCAM(model=model, target_layers=[model.layer4[-1]])
heatmap = cam(input_tensor=tensor)[0]
vis = show_cam_on_image(img_np, heatmap, use_rgb=True)

plt.imshow(vis); plt.axis("off")
plt.savefig(f"{OUT_DIR}/gradcam.jpg", dpi=300)
plt.close()

wandb.log({"gradcam": wandb.Image(f"{OUT_DIR}/gradcam.jpg")})

wandb.finish()


Running on: cuda


FLOPs,▁▁▁
Parameters,▁▁▁
brightness_drop,▁
epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
macro_f1,▁▁▁
occlusion_drop,▁
robust_acc_brightness,▁
robust_acc_clean,▁
robust_acc_occlusion,▁
train_acc,▁▂▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇█████
+3,...


Classes: 116
Training images (pretrained): 10000
Validation images:            2000
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 191MB/s]
100%|██████████| 313/313 [01:23<00:00,  3.73it/s]


Epoch 1: TrainAcc=0.852 ValAcc=0.998


...

100%|██████████| 313/313 [01:14<00:00,  4.19it/s]


Epoch 19: TrainAcc=1.000 ValAcc=1.000


100%|██████████| 313/313 [01:14<00:00,  4.20it/s]


Epoch 20: TrainAcc=1.000 ValAcc=1.000
Macro F1: 1.0


epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
macro_f1,▁
num_classes,▁
train_acc,▁███████████████████
train_images,▁
train_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_acc,▇█████▁█████████████
val_images,▁
val_loss,█▂▁▁▁▁▇▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,20
macro_f1,1


In [ ]:
import os, random
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt
import numpy as np
import wandb


VAL_PATH = "/content/fruits_dataset/fruits-360_original-size/fruits-360-original-size/Validation"
WEIGHTS_PATH = "/content/pretrained/best_model_pretrained.pth"
NUM_VAL_TEST = 3000
BATCH_SIZE = 32

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Running on:", device)

os.makedirs("/content/noaugment", exist_ok=True)


imagenet_norm = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std =[0.229, 0.224, 0.225]
)

clean_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    imagenet_norm
])

brightness_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ColorJitter(brightness=0.5),
    transforms.ToTensor(),
    imagenet_norm
])

occlusion_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.RandomErasing(p=1.0, scale=(0.1, 0.2)),
    imagenet_norm
])


tmp_ds = datasets.ImageFolder(VAL_PATH, transform=clean_transform)
num_classes = len(tmp_ds.classes)
print(f"Number of classes: {num_classes}")


model = models.resnet18(weights=None)  # architecture only
model.fc = nn.Linear(model.fc.in_features, num_classes)

model.load_state_dict(torch.load(WEIGHTS_PATH, map_location=device))
model = model.to(device)
model.eval()

print(f"✅ Loaded pretrained ResNet-18 weights from {WEIGHTS_PATH}")


indices = random.sample(range(len(tmp_ds)), NUM_VAL_TEST)

clean_ds  = Subset(datasets.ImageFolder(VAL_PATH, transform=clean_transform), indices)
bright_ds = Subset(datasets.ImageFolder(VAL_PATH, transform=brightness_transform), indices)
occl_ds   = Subset(datasets.ImageFolder(VAL_PATH, transform=occlusion_transform), indices)

print(f"Validation images tested (robustness): {len(clean_ds)}")

clean_loader  = DataLoader(clean_ds,  batch_size=BATCH_SIZE, shuffle=False)
bright_loader = DataLoader(bright_ds, batch_size=BATCH_SIZE, shuffle=False)
occl_loader   = DataLoader(occl_ds,   batch_size=BATCH_SIZE, shuffle=False)

=
def eval_accuracy(loader):
    correct, total = 0, 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            preds = model(x).argmax(1)
            correct += (preds == y).sum().item()
            total += y.size(0)
    return correct / total


acc_clean  = eval_accuracy(clean_loader)
acc_bright = eval_accuracy(bright_loader)
acc_occl   = eval_accuracy(occl_loader)

drop_bright = acc_clean - acc_bright
drop_occl   = acc_clean - acc_occl

print("\n📊 ROBUSTNESS RESULTS (PRETRAINED)")
print(f"Clean accuracy:        {acc_clean:.4f}")
print(f"Brightness accuracy:   {acc_bright:.4f}")
print(f"Occlusion accuracy:    {acc_occl:.4f}")
print(f"Brightness drop:       {drop_bright:.4f}")
print(f"Occlusion drop:        {drop_occl:.4f}")


labels = ["Clean", "Brightness", "Occlusion"]
accuracies = [acc_clean, acc_bright, acc_occl]

plt.figure(figsize=(6,4))
plt.bar(labels, accuracies)
plt.ylim(0, 1.0)
plt.ylabel("Accuracy")
plt.title("Pretrained ResNet-18 Robustness")
plt.tight_layout()
plt.savefig("/content/pretrained/robustness_accuracy_pretrained.jpg", dpi=300)
plt.close()

plt.figure(figsize=(6,4))
plt.bar(["Brightness Drop", "Occlusion Drop"], [drop_bright, drop_occl])
plt.ylabel("Accuracy Drop")
plt.title("Pretrained Robustness Degradation")
plt.tight_layout()
plt.savefig("/content/pretrained/robustness_drop_pretrained.jpg", dpi=300)
plt.close()

print("📁 Saved pretrained robustness plots in /content/noaugment/")


try:
    wandb.log({
        "pretrained_acc_clean": acc_clean,
        "pretrained_acc_brightness": acc_bright,
        "pretrained_acc_occlusion": acc_occl,
        "pretrained_brightness_drop": drop_bright,
        "pretrained_occlusion_drop": drop_occl,
        "pretrained_robustness_accuracy_plot": wandb.Image(
            "/content/pretrained/robustness_accuracy_pretrained.jpg"
        ),
        "pretrained_robustness_drop_plot": wandb.Image(
            "/content/pretrainde/robustness_drop_pretrained.jpg"
        )
    })
    print("📈 Logged pretrained robustness metrics to W&B")
except:
    print("ℹ️ W&B not active — skipped logging")


Running on: cuda
Number of classes: 116
✅ Loaded pretrained ResNet-18 weights from /content/pretrained/best_model_pretrained.pth
Validation images tested (robustness): 3000

📊 ROBUSTNESS RESULTS (PRETRAINED)
Clean accuracy:        1.0000
Brightness accuracy:   0.9953
Occlusion accuracy:    0.9973
Brightness drop:       0.0047
Occlusion drop:        0.0027
📁 Saved pretrained robustness plots in /content/noaugment/
ℹ️ W&B not active — skipped logging


In [27]:
from torchviz import make_dot
import torch
from torchvision import models

model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
dummy = torch.randn(1, 3, 224, 224)

dot = make_dot(model(dummy), params=dict(model.named_parameters()))
dot.format = "png"
dot.render("/content/outputs/resnet18_architecture")

print("Saved to /content/outputs/resnet18_architecture.png")


Saved to /content/outputs/resnet18_architecture.png
